In [ ]:
import torch
import sklearn
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split


In [ ]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

In [ ]:
X_train

array([[   3.5214    ,   15.        ,    3.04994451, ...,    1.60599334,
          37.63      , -122.43      ],
       [   5.3275    ,    5.        ,    6.49005964, ...,    3.44333996,
          33.69      , -117.39      ],
       [   3.1       ,   29.        ,    7.54237288, ...,    2.25084746,
          38.44      , -122.98      ],
       ...,
       [   1.5561    ,   35.        ,    4.13993174, ...,    3.27645051,
          34.11      , -117.29      ],
       [   2.1055    ,   52.        ,    5.17849462, ...,    2.92688172,
          36.76      , -119.79      ],
       [   5.6022    ,   31.        ,    6.21971253, ...,    2.53798768,
          38.04      , -122.2       ]])

In [ ]:
X_train = torch.FloatTensor(X_train)

In [ ]:
X_train

tensor([[   3.5214,   15.0000,    3.0499,  ...,    1.6060,   37.6300,
         -122.4300],
        [   5.3275,    5.0000,    6.4901,  ...,    3.4433,   33.6900,
         -117.3900],
        [   3.1000,   29.0000,    7.5424,  ...,    2.2508,   38.4400,
         -122.9800],
        ...,
        [   1.5561,   35.0000,    4.1399,  ...,    3.2765,   34.1100,
         -117.2900],
        [   2.1055,   52.0000,    5.1785,  ...,    2.9269,   36.7600,
         -119.7900],
        [   5.6022,   31.0000,    6.2197,  ...,    2.5380,   38.0400,
         -122.2000]])

In [ ]:
X_valid = torch.FloatTensor(X_valid)
X_test = torch.FloatTensor(X_test)

means = X_train.mean(dim=0, keepdim=True)
stds = X_train.std(dim=0, keepdim=True)

X_train = (X_train - means) / stds
X_valid = (X_valid - means) / stds
X_test = (X_test - means) / stds

In [ ]:
X_train.shape

torch.Size([11610, 8])

In [ ]:
y_train

array([1.442, 1.687, 1.621, ..., 0.68 , 0.613, 1.97 ])

In [ ]:
y_train = torch.FloatTensor(y_train).reshape(-1,1)
y_valid = torch.FloatTensor(y_valid).reshape(-1,1)
y_test = torch.FloatTensor(y_test).reshape(-1,1)

In [ ]:
y_train

tensor([[1.4420],
        [1.6870],
        [1.6210],
        ...,
        [0.6800],
        [0.6130],
        [1.9700]])

In [ ]:
torch.manual_seed(42)

In [ ]:
n_features = X_train.shape[1] # this returns just a number 8

In [ ]:
w = torch.randn((n_features, 1), requires_grad=True)
b = torch.tensor(0., requires_grad=True)

In [ ]:
learning_rate = 0.4
n_epochs = 20

for epoch in range(n_epochs):
    y_pred = X_train @ w + b
    loss = ((y_pred-y_train)**2).mean()
    loss.backward()
    with torch.no_grad():
        b -=learning_rate* b.grad
        w -=learning_rate* w.grad
        b.grad.zero_()
        w.grad.zero_()
    print(f"Epoch {epoch+1}/{n_epochs}, Loss: {loss.item()}")

Epoch 1/20, Loss: 16.158456802368164
Epoch 2/20, Loss: 4.8793745040893555
Epoch 3/20, Loss: 2.255225419998169
Epoch 4/20, Loss: 1.3307636976242065
Epoch 5/20, Loss: 0.9680693745613098
Epoch 6/20, Loss: 0.8142675757408142
Epoch 7/20, Loss: 0.7417045831680298
Epoch 8/20, Loss: 0.7020700573921204
Epoch 9/20, Loss: 0.6765917539596558
Epoch 10/20, Loss: 0.6577963829040527
Epoch 11/20, Loss: 0.6426151394844055
Epoch 12/20, Loss: 0.6297222971916199
Epoch 13/20, Loss: 0.6184941530227661
Epoch 14/20, Loss: 0.6085968017578125
Epoch 15/20, Loss: 0.5998216271400452
Epoch 16/20, Loss: 0.592018723487854
Epoch 17/20, Loss: 0.5850691795349121
Epoch 18/20, Loss: 0.578873336315155
Epoch 19/20, Loss: 0.573345422744751
Epoch 20/20, Loss: 0.5684100389480591


In [ ]:
X_new = X_test[:3]
X_new

tensor([[-1.1578, -0.2867, -0.4955, -0.1662, -0.0295,  0.3890,  0.1937,  0.2870],
        [-0.7125,  0.1088, -0.1633,  0.2016,  0.1284, -0.1182, -0.2372,  0.0622],
        [-0.2156,  1.8491, -0.5798,  0.1853, -0.1043, -0.6769,  1.0089, -1.4271]])

In [ ]:
with torch.no_grad():
    y_pred = X_new @ w + b
y_pred

tensor([[0.8916],
        [1.6480],
        [2.6577]])

In [ ]:
import torch.nn as nn
torch.manual_seed(42)
model = nn.Linear(in_features=n_features, out_features=1)

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
mse= nn.MSELoss()

In [ ]:
def train_bgd(model, optimizer, criterion, X_train, y_train, n_epochs):
  for epoch in range(n_epochs):
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    print(f"Epoch {epoch+1}/{n_epochs}, Loss: {loss.item()}")

In [ ]:
train_bgd(model, optimizer, mse, X_train, y_train, n_epochs)

Epoch 1/20, Loss: 4.3378496170043945
Epoch 2/20, Loss: 0.7802939414978027
Epoch 3/20, Loss: 0.6253842115402222
Epoch 4/20, Loss: 0.6060433983802795
Epoch 5/20, Loss: 0.5956299304962158
Epoch 6/20, Loss: 0.587356686592102
Epoch 7/20, Loss: 0.5802990794181824
Epoch 8/20, Loss: 0.5741382241249084
Epoch 9/20, Loss: 0.5687101483345032
Epoch 10/20, Loss: 0.5639079809188843
Epoch 11/20, Loss: 0.5596511363983154
Epoch 12/20, Loss: 0.5558737516403198
Epoch 13/20, Loss: 0.5525194406509399
Epoch 14/20, Loss: 0.5495392084121704
Epoch 15/20, Loss: 0.5468900203704834
Epoch 16/20, Loss: 0.5445339679718018
Epoch 17/20, Loss: 0.5424376726150513
Epoch 18/20, Loss: 0.5405716300010681
Epoch 19/20, Loss: 0.5389097332954407
Epoch 20/20, Loss: 0.5374288558959961


In [ ]:
with torch.no_grad():
  y_pred = model(X_new)

In [ ]:
y_pred

tensor([[0.8061],
        [1.7116],
        [2.6973]])

In [ ]:
torch.manual_seed(42)
model = nn.Sequential(
    nn.Linear(n_features, 50),
    nn.ReLU(),
    nn.Linear(50, 40),
    nn.ReLU(),
    nn.Linear(40,1)
)

learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
mse= nn.MSELoss()
train_bgd(model, optimizer, mse, X_train, y_train, n_epochs)

Epoch 1/20, Loss: 5.045480251312256
Epoch 2/20, Loss: 2.0523123741149902
Epoch 3/20, Loss: 1.0039883852005005
Epoch 4/20, Loss: 0.8570138216018677
Epoch 5/20, Loss: 0.7740675210952759
Epoch 6/20, Loss: 0.7225848436355591
Epoch 7/20, Loss: 0.6893726587295532
Epoch 8/20, Loss: 0.6669032573699951
Epoch 9/20, Loss: 0.650773823261261
Epoch 10/20, Loss: 0.6383934020996094
Epoch 11/20, Loss: 0.6281994581222534
Epoch 12/20, Loss: 0.6193399429321289
Epoch 13/20, Loss: 0.6113173365592957
Epoch 14/20, Loss: 0.6038705706596375
Epoch 15/20, Loss: 0.5968307852745056
Epoch 16/20, Loss: 0.5901118516921997
Epoch 17/20, Loss: 0.583646833896637
Epoch 18/20, Loss: 0.5774063467979431
Epoch 19/20, Loss: 0.5713554620742798
Epoch 20/20, Loss: 0.5654447674751282


In [ ]:
from torch.utils.data import DataLoader, TensorDataset
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)

In [ ]:
torch.manual_seed(42)
model = nn.Sequential(
    nn.Linear(n_features, 50),
    nn.ReLU(),
    nn.Linear(50, 40),
    nn.ReLU(),
    nn.Linear(40,1)
)

learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
mse= nn.MSELoss()

In [ ]:
device = 'cuda'
model = model.to(device, non_blocking=True)

In [ ]:
# def train(X_train, y_train, optimizer, criterion, model, n_epochs, batch_size)
#   for epoch in range(n_epochs):
#     y_pred = model(X_train)
#     loss = criterion(y_pred, y_train)
#     model.backward()
#     optimizer.step()
#     optimizer.zero_grad()
#     print(f"Epoch {epoch+1}/{n_epochs}, Loss: {loss.item()}")

# train(train_loader(X_train, y_train), optimizer, mse, learning_rate, model)

def train(model, optimizer, criterion, train_loader, n_epochs):
  model.train()
  for epoch in range(n_epochs):
      total_loss = 0.
      for X_batch, y_batch in train_loader:
          X_batch, y_batch = X_batch.to(device), y_batch.to(device)
          y_pred = model(X_batch)
          loss = criterion(y_pred, y_batch)
          total_loss += loss.item()
          loss.backward()
          optimizer.step()
          optimizer.zero_grad()
      mean_loss = total_loss / len(train_loader)
      print(f"Epoch {epoch+1}/{n_epochs}, Loss: {mean_loss}")

train(model, optimizer, mse, train_loader, n_epochs)

Epoch 1/20, Loss: 0.5220467695355744
Epoch 2/20, Loss: 0.410424821961353
Epoch 3/20, Loss: 0.3752931169036663
Epoch 4/20, Loss: 0.35843538558598065
Epoch 5/20, Loss: 0.3523593365160887
Epoch 6/20, Loss: 0.34400911364427283
Epoch 7/20, Loss: 0.32950141284994516
Epoch 8/20, Loss: 0.3255853181904998
Epoch 9/20, Loss: 0.32395407992617814
Epoch 10/20, Loss: 0.3184097018862559
Epoch 11/20, Loss: 0.3226995343758055
Epoch 12/20, Loss: 0.31286822600141373
Epoch 13/20, Loss: 0.30840102952620213
Epoch 14/20, Loss: 0.3014247566256313
Epoch 15/20, Loss: 0.29947117149583563
Epoch 16/20, Loss: 0.29599220192793646
Epoch 17/20, Loss: 0.29323662117575156
Epoch 18/20, Loss: 0.2926799827986512
Epoch 19/20, Loss: 0.29342265566682685
Epoch 20/20, Loss: 0.29593946171036767


In [ ]:
def evaluate(model, data_loader, metric_fn, aggregate_fn=torch.mean):
  model.eval()
  metrics = []
  with torch.no_grad():
    for X_batch, y_batch in data_loader:
      X_batch, y_batch = X_batch.to(device), y_batch.to(device)
      y_pred = model(X_batch)
      metric = metric_fn(y_pred, y_batch)
      metrics.append(metric)
  return aggregate_fn(torch.tensor(metrics))

In [ ]:
valid_dataset = TensorDataset(X_valid, y_valid)
valid_loader = DataLoader(valid_dataset, batch_size=32)
valid_mse = evaluate(model, valid_loader, mse)
valid_mse

tensor(0.3005)

In [ ]:
import torchmetrics
def evaluate_tm(model, data_loader, metric):
  model.eval()
  metrics = []
  with torch.no_grad():
    for X_batch, y_batch in data_loader:
      X_batch, y_batch = X_batch.to(device), y, batch.to(device)
      y_pred = model(X_batch)
      metric.update(y_pred, y_batch)
  return metric.compute()

In [ ]:
rmse = torchmetrics.MeanSquaredError(squared=False).to(device)
evaluate_tm(model, valid_loader, rmse)